In [1]:
#Importando os pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Lendo os dados
df = pd.read_csv('./data/winequality-red.csv')

In [150]:
df.head()
# df.columns
# df.dtypes # float64 e int64
# df.shape # (1599, 12)
# df.isna().sum() # 0 valores faltantes
# df.info()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [21]:
#Criando o X e y
X = df.drop(columns = ['quality'])
y = df['quality']

In [161]:
print(df.shape)
print(X.shape)
print(y.shape)

(1599, 12)
(1599, 11)
(1599,)


In [27]:
# Separando dados em treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [51]:
# Normalização

#StandardScaler
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_train_norm = std.fit_transform(X_train)
X_test_norm = std.transform(X_test)

In [59]:
# Normalização

# Maximos Absolutos
from sklearn.preprocessing import MaxAbsScaler

maxabs = MaxAbsScaler()
X_train_norm = maxabs.fit_transform(X_train)
X_test_norm = maxabs.transform(X_test)

In [64]:
# Seleção de Feature

# Seleção de feature univariada
from sklearn.feature_selection import VarianceThreshold

#Instanciando o objeto
var = VarianceThreshold()

#Ajustando aos dados
var.fit(X_train)

#Verificando a variancia e as variaveis selecionadas
pd.DataFrame({'colunas': X_train.columns,'variancia': var.variances_, 'selecionada': var.get_support()})

,colunas,variancia,selecionada
0,fixed acidity,2.934824,True
1,volatile acidity,0.033102,True
2,citric acid,0.038183,True
3,residual sugar,2.036592,True
4,chlorides,0.002220,True
5,free sulfur dioxide,105.443629,True
6,total sulfur dioxide,1090.447715,True
7,density,0.000003,True
8,pH,0.023689,True
9,sulphates,0.029641,True


In [112]:
# Seleção de Feature

#Seleção de features no sklearn
from sklearn.feature_selection import RFECV

#Instanciando o objeto
rfe = RFECV(LinearRegression())

#Ajustando aos dados e transformando o dataframe
X_train_rfe = rfe.fit_transform(X_train, y_train)

/home/leandro/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [113]:
#Verificando a diferença entre os dataframes
print(X_train_rfe.shape, X_train.shape)

# Plotando o score e as features selecionadas
df_rfe = pd.DataFrame({'colunas': X_train.columns, 'score': rfe.grid_scores_, 'selecionada': rfe.get_support()})

df_rfe

(1119, 11) (1119, 11)


,colunas,score,selecionada
0,fixed acidity,0.028138,True
1,volatile acidity,0.172265,True
2,citric acid,0.174180,True
3,residual sugar,0.218261,True
4,chlorides,0.253567,True
5,free sulfur dioxide,0.247901,True
6,total sulfur dioxide,0.327418,True
7,density,0.322637,True
8,pH,0.322586,True
9,sulphates,0.322304,True


In [114]:
colunas = list(df_rfe[df_rfe['selecionada']]['colunas'])
colunas

X_test_rfe = X_test[colunas]

X_test_rfe.shape

(480, 11)

In [122]:
# Balanceamento

#Importando os pacotes
# conda install -c conda-forge imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

#Instanciando o objeto
ros = RandomOverSampler()

#Aplicando o resample
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

#Somando os casos com 1 em nosso target
print(y_train.sum())

#Somando os casos com 1 na base
print(y_train_ros.sum())

6291
16038


In [133]:
# Balanceamento

#Importando os pacotes
from imblearn.under_sampling import RandomUnderSampler

#Instaciando o objeto
rus = RandomUnderSampler()

#Aplicando o resample
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

#Printando o shape da base de dados
print(X_train.shape)

#Shape apos transformação
print(X_train_rus.shape)

(1119, 11)
(54, 11)


In [138]:
# Modelo

#Regressão Linear | Importando os pacotes e instanciando o objeto
from sklearn.linear_model import LinearRegression

#Instanciando o objeto
model = LinearRegression()

#Ajustando o modelo
model.fit(X_train, y_train)
# model.fit(X_train_norm, y_train)
# model.fit(X_train_rfe, y_train)
# model.fit(X_train_ros, y_train_ros)
# model.fit(X_train_rus, y_train_rus)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [139]:
#Realizando a predição
y_pred = model.predict(X_test)
# y_pred = model.predict(X_test_norm)
# y_pred = model.predict(X_test_rfe)

In [140]:
#Criando o dataframe com os resultados
prr = pd.DataFrame({'predito' : y_pred, 'real': y_test, 'residuo': y_pred - y_test})
prr

,predito,real,residuo
803,5.356763,6,-0.643237
124,5.090715,5,0.090715
350,5.625538,6,-0.374462
682,5.448861,5,0.448861
1326,5.744784,6,-0.255216
...,...,...,...
1468,5.597986,7,-1.402014
495,6.105900,8,-1.894100
1325,5.744784,6,-0.255216
514,6.101798,7,-0.898202


In [141]:
# Sklearn

#Erro medio absoluto
from sklearn.metrics import mean_absolute_error

#Erro medio absoluto
mean_absolute_error(y_pred, y_test)

0.5133956082451123

In [144]:
# Auto ML
#Importando os pacotes
from tpot import TPOTRegressor

#Instanciando o objeto
tpot = TPOTRegressor(population_size = 3, generations = 2, verbosity = 2)

#Ajustando os dados
# Fazendo o train test split com o y para regressao
# X_train, X_test, y_train, y_test = train_test_split(X, y_reg, random_state = 42, test_size = 0.3)

#Ajustando aos dados
tpot.fit(X_train,y_train)

Optimization Progress: 100%|██████████| 9/9 [00:06<00:00,  1.34pipeline/s]Generation 2 - Current best internal CV score: -0.41336411081517044

Best pipeline: AdaBoostRegressor(input_matrix, learning_rate=0.5, loss=linear, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=2,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=3,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [148]:
#Criando o modelo indicado pelo tpot
from sklearn.pipeline import make_pipeline, make_union
from sklearn.ensemble import AdaBoostRegressor

exported_pipeline = make_pipeline(AdaBoostRegressor(learning_rate=0.5, loss='linear', n_estimators=100))
    
exported_pipeline.fit(X_train, y_train)
y_pred_tpot = exported_pipeline.predict(X_test)

In [149]:
# Sklearn

#Erro medio absoluto
from sklearn.metrics import mean_absolute_error

#Erro medio absoluto
mean_absolute_error(y_pred_tpot, y_test)

0.526728857597664